In [13]:
import numpy as np
import os
import pandas as pd
# import psycopg
import requests
from sqlalchemy import create_engine, DateTime


In [2]:
os.environ["PG_HOST"] = "db"
os.environ["PG_DATABASE"] = "weather"
os.environ["PG_USER"] = "postgres"
os.environ["PG_PASSWORD"] = "postgres"
os.environ["API_KEY"] = "MD8gOfRQx4yZ0lS8CfXv6OkVtBLCIdw8"



In [3]:
host = os.environ["PG_HOST"]
db = os.environ["PG_DATABASE"]
user = os.environ["PG_USER"]
pw = os.environ["PG_PASSWORD"]
api_key = os.environ["API_KEY"]

In [4]:

locations = np.array([
    ('25.8600','-97.4200'),
    ('25.9000','-97.5200'),
    ('25.9000','-97.4800'),
    ('25.9000','-97.4400'),
    ('25.9000','-97.4000'),
    ('25.9200','-97.3800'),
    ('25.9400','-97.5400'),
    ('25.9400','-97.5200'),
    ('25.9400','-97.4800'),
    ('25.9400','-97.4400'),
])

locations_df = pd.DataFrame.from_records(locations, columns=["lat", "lon"])
locations_df.insert(0, 'id', range(len(locations_df)))
locations_df = locations_df.head(2)
display(locations_df)


,id,lat,lon
0,0,25.8600,-97.4200
1,1,25.9000,-97.5200


In [5]:
connect = f"postgresql+psycopg2://{user}:{pw}@{host}:5432/{db}"
engine = create_engine(connect)
locations_df.to_sql(
    'locations', 
    con=engine, 
    index=False, 
    if_exists='replace'
)


2

In [15]:
for ix, row in locations_df.iterrows():
    lat = str(row["lat"])
    lon = str(row["lon"])
    if_exists = 'replace' if ix == 0 else 'append'

    url = f"https://api.tomorrow.io/v4/weather/realtime?location={lat}%2C%20{lon}&units=metric&apikey={api_key}"
    
    headers = {"accept": "application/json"}
    
    response = requests.get(url, headers=headers)
    df = pd.json_normalize(response.json()["data"])
    df.insert(0, 'lat', lat)
    df.insert(1, 'lon', lon)
    display(df)

    
    df.to_sql(
        'realtime', 
        con=engine, 
        index=False, 
        if_exists=if_exists
    )
    print("to_sql() done (sqlalchemy)")

    url = "https://api.tomorrow.io/v4/timelines?apikey=" + api_key
    
    payload = {
        "location": f"{lat},{lon}", 
        "fields": ["temperature", "windSpeed"],
        "units": "metric",
        "timesteps": ["1h"],
        "startTime": "nowMinus1d",
        "endTime": "nowPlus5d"
    }
    headers = {
        "accept": "application/json",
        "Accept-Encoding": "gzip",
        "content-type": "application/json"
    }
    
    response = requests.post(url, json=payload, headers=headers)
    response_json = response.json()
    timelines = response_json["data"]["timelines"]
    timeline = timelines[0]
    intervals = timeline["intervals"]
    df = pd.DataFrame(intervals)
    print(df)
    df = pd.json_normalize(intervals)
    print(df)
    df.insert(0, 'lat', lat)
    df.insert(1, 'lon', lon)
    df

    df.to_sql(
        'timeline', 
        con=engine, 
        index=False, 
        if_exists=if_exists,
        dtype={"startTime": DateTime()}
    )
    print("to_sql() done (sqlalchemy)")
    
    # url = "https://api.tomorrow.io/v4/weather/realtime?location=42.3478%2C%20-71.0466&units=metric&apikey" + api_key
    
    # json_str = json.dumps(response_json["data"]["timelines"], indent=2)
    
    # print(json_str)
    # print(response.json())

,lat,lon,time,values.cloudBase,values.cloudCeiling,values.cloudCover,values.dewPoint,values.freezingRainIntensity,values.humidity,values.precipitationProbability,...,values.snowIntensity,values.temperature,values.temperatureApparent,values.uvHealthConcern,values.uvIndex,values.visibility,values.weatherCode,values.windDirection,values.windGust,values.windSpeed
0,25.8600,-97.4200,2024-03-25T00:01:00Z,0.46,None,41,19.63,0,80,0,...,0,23.19,23.19,0,0,16,1101,157,14.19,8.31


to_sql() done (sqlalchemy)
                startTime                                      values
0    2024-03-24T00:00:00Z    {'temperature': 20.5, 'windSpeed': 4.88}
1    2024-03-24T01:00:00Z      {'temperature': 19, 'windSpeed': 3.38}
2    2024-03-24T02:00:00Z    {'temperature': 18.38, 'windSpeed': 2.5}
3    2024-03-24T03:00:00Z      {'temperature': 18, 'windSpeed': 2.88}
4    2024-03-24T04:00:00Z   {'temperature': 18.13, 'windSpeed': 2.81}
..                    ...                                         ...
140  2024-03-29T20:00:00Z  {'temperature': 25.44, 'windSpeed': 10.32}
141  2024-03-29T21:00:00Z  {'temperature': 25.29, 'windSpeed': 10.19}
142  2024-03-29T22:00:00Z   {'temperature': 24.44, 'windSpeed': 9.57}
143  2024-03-29T23:00:00Z   {'temperature': 23.59, 'windSpeed': 8.94}
144  2024-03-30T00:00:00Z   {'temperature': 22.74, 'windSpeed': 8.32}

[145 rows x 2 columns]
                startTime  values.temperature  values.windSpeed
0    2024-03-24T00:00:00Z               20.50

,lat,lon,time,values.cloudBase,values.cloudCeiling,values.cloudCover,values.dewPoint,values.freezingRainIntensity,values.humidity,values.precipitationProbability,...,values.snowIntensity,values.temperature,values.temperatureApparent,values.uvHealthConcern,values.uvIndex,values.visibility,values.weatherCode,values.windDirection,values.windGust,values.windSpeed
0,25.9000,-97.5200,2024-03-25T00:01:00Z,0.65,None,29,19.19,0,73,0,...,0,24.31,24.31,0,0,16,1100,156.69,13.69,7.5


to_sql() done (sqlalchemy)
                startTime                                      values
0    2024-03-24T00:00:00Z   {'temperature': 21.81, 'windSpeed': 4.69}
1    2024-03-24T01:00:00Z   {'temperature': 20.19, 'windSpeed': 3.38}
2    2024-03-24T02:00:00Z   {'temperature': 19.19, 'windSpeed': 2.38}
3    2024-03-24T03:00:00Z   {'temperature': 18.63, 'windSpeed': 2.88}
4    2024-03-24T04:00:00Z   {'temperature': 18.69, 'windSpeed': 2.88}
..                    ...                                         ...
140  2024-03-29T20:00:00Z   {'temperature': 25.84, 'windSpeed': 10.3}
141  2024-03-29T21:00:00Z  {'temperature': 25.65, 'windSpeed': 10.18}
142  2024-03-29T22:00:00Z   {'temperature': 24.73, 'windSpeed': 9.61}
143  2024-03-29T23:00:00Z   {'temperature': 23.81, 'windSpeed': 9.04}
144  2024-03-30T00:00:00Z   {'temperature': 22.89, 'windSpeed': 8.46}

[145 rows x 2 columns]
                startTime  values.temperature  values.windSpeed
0    2024-03-24T00:00:00Z               21.81